# Домашняя работа по теме "Модель BERT и GPT"

__Задание__

Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза

(дополнительно необязательная задача)на выбор взять https://huggingface.co/datasets/sberquad https://huggingface.co/datasets/blinoff/medical_qa_ru_data натренировать любую модель для вопросно ответной системы как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER


In [1]:
!pip install datasets transformers==4.28.0
!pip install Sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import datasets
from datasets import load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers import DataCollatorWithPadding

In [3]:
corpus = datasets.load_dataset('merionum/ru_paraphraser', data_files='plus.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'paraphrases'],
        num_rows: 1725393
    })
})

In [5]:
corpus['train']

Dataset({
    features: ['id', 'paraphrases'],
    num_rows: 1725393
})

In [6]:
corpus['train'][:1]

{'id': ['0'],
 'paraphrases': [['Птичий праздник: Что такое "куриное рождество" и кто его отмечает',
   'Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем']]}

In [7]:
corpus['train']['paraphrases'][0]

['Птичий праздник: Что такое "куриное рождество" и кто его отмечает',
 'Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем']

In [8]:
corpus['train']['paraphrases'][0][0]

'Птичий праздник: Что такое "куриное рождество" и кто его отмечает'

# Используем предобученную модель: cointegrated/rut5-base-paraphraser

### https://huggingface.co/cointegrated/rut5-base-paraphraser

In [9]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda();
model.eval();

def paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

print(paraphrase('У Лукоморья дуб зелёный, златая цепь на дубе том.'))

Лукоморье дуб зеленый, златый дуб на том дубе.


# Используем предобученную модель: cointegrated/rubert-base-cased-dp-paraphrase-detection
### https://huggingface.co/cointegrated/rubert-base-cased-dp-paraphrase-detection

In [10]:
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
tokenizer = BertTokenizer.from_pretrained(model_name)

def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()

    p = proba[0]  # p(non-paraphrase), p(paraphrase)
    if p[1] > 0.5:
        paraphrase = "paraphrase"
    else:
        paraphrase = "non"

    return [text1, text2, p[0], p[1], paraphrase]


print(compare_texts('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода'))

print(compare_texts('Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась'))


['Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода', 0.70562273, 0.29437727, 'non']
['Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась', 0.16524385, 0.83475614, 'paraphrase']


In [11]:
def compare(text1, text2):
    """  """

    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()

    p = proba[0]  # p(non-paraphrase), p(paraphrase)
    paraphrase = "non"

    if p[1] > 0.5:
        paraphrase = "paraphrase"

    return [text1, text2, p[0], p[1], paraphrase]

In [12]:
import pandas as pd

pd.set_option('display.max_colwidth', 100)

In [13]:
result = []

result.append(compare('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода'))
result.append(compare('Цены на нефть восстанавливаются', 'Парламент Словакии поблагодарил народы'))
result.append(compare('Сегодня на улице хорошая погода', 'Сегодня на улице прекрасная погода'))
n=5
for i in range(n):
  text1 = corpus['train']['paraphrases'][i][0]
  text2 =corpus['train']['paraphrases'][i][1]
  result.append(compare(text1, text2))



col_names =  ['текст1', 'текст2', 'вероятность non-paraphrase', 'вероятность paraphrase', 'paraphrase']
df = pd.DataFrame(result, columns=col_names)
df.head(10)

,текст1,текст2,вероятность non-paraphrase,вероятность paraphrase,paraphrase
0,Сегодня на улице хорошая погода,Сегодня на улице отвратительная погода,0.705623,0.294377,non
1,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы,0.944955,0.055045,non
2,Сегодня на улице хорошая погода,Сегодня на улице прекрасная погода,0.035242,0.964758,paraphrase
3,"Птичий праздник: Что такое ""куриное рождество"" и кто его отмечает",Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем,0.157879,0.842121,paraphrase
4,В Индии 5 млн женщин выстроились стеной в 620 км ради равенства (Фото),"Индийские женщины выстроили живую стену длиной 620 километров, требуя равенства",0.018209,0.981791,paraphrase
5,"Томас Бах: ""Олимпийский комитет России понес достаточное наказание""",Президент МОК Бах: Россия уже достаточно наказана за допинг,0.028516,0.971484,paraphrase
6,Россия не представила WADA данные по пробам в срок и может быть наказана,WADA: Россия не предоставила нам доступ к допинг-пробам,0.058580,0.941420,paraphrase
7,Антидопинговое агентство США призвало ВАДА отстранить российских спортсменов от соревнований,Комиссия спортсменов WADA потребовала отстранить РУСАДА,0.428032,0.571968,paraphrase
